In [1]:
from glob import glob

In [3]:
all_dark_fits = glob("./DarkMeasurements/dark*.fits")
print(all_dark_fits)

['./DarkMeasurements\\dark_0000.fits', './DarkMeasurements\\dark_0001.fits', './DarkMeasurements\\dark_0002.fits', './DarkMeasurements\\dark_0003.fits', './DarkMeasurements\\dark_0004.fits', './DarkMeasurements\\dark_0005.fits', './DarkMeasurements\\dark_0006.fits', './DarkMeasurements\\dark_0007.fits', './DarkMeasurements\\dark_0008.fits', './DarkMeasurements\\dark_0009.fits', './DarkMeasurements\\dark_0010.fits', './DarkMeasurements\\dark_0011.fits', './DarkMeasurements\\dark_0012.fits', './DarkMeasurements\\dark_0013.fits', './DarkMeasurements\\dark_0014.fits', './DarkMeasurements\\dark_0015.fits', './DarkMeasurements\\dark_0016.fits', './DarkMeasurements\\dark_0017.fits', './DarkMeasurements\\dark_0018.fits', './DarkMeasurements\\dark_0019.fits', './DarkMeasurements\\dark_0020.fits', './DarkMeasurements\\dark_0021.fits', './DarkMeasurements\\dark_0022.fits', './DarkMeasurements\\dark_0023.fits', './DarkMeasurements\\dark_0024.fits', './DarkMeasurements\\dark_0025.fits', './DarkMeas